 Importing necessary packages

In [1]:
import numpy as np
import torch
import data_processing
from data_processing import next_batch
from network_module import NeuralNet,NeuralNet_multilayerencode
import torch.nn as nn
import torch.optim as optim
from torch.autograd import Variable


# Above data_prrocessing and network_module are self made module for this task and is available in the repository

Taking the same dataset and preprocessing that I have applied earlier for the same task using keras.

In [2]:
dataread = data_processing.Datareading(dataframe = 'dataset.csv')
Independent_data,Response = dataread.dataprocessed()
scaler,df  = data_processing.__Processing__(df = Independent_data,process='min-max')


In [4]:
df[:5,:]

array([[  0.00000000e+00,   8.44035463e-02,   0.00000000e+00,
          7.76475734e-05,   2.35309258e-02,   1.65677595e-02,
          1.83312544e-02,   1.84915258e-03,   1.71414480e-02,
          3.94041229e-02,   2.63142874e-02,   2.46140780e-02,
          2.49297844e-02,   8.97226191e-02],
       [  1.76149375e-04,   8.35840012e-02,   0.00000000e+00,
          0.00000000e+00,   2.30505344e-02,   1.40488170e-02,
          1.93644116e-02,   6.14204447e-03,   1.87181926e-02,
          3.95184737e-02,   2.60509003e-02,   2.44917674e-02,
          2.51390123e-02,   9.50835235e-02],
       [  3.52298749e-04,   8.19610704e-02,   3.09318131e-04,
          7.66195632e-05,   2.23002404e-02,   1.35873757e-02,
          1.72959379e-02,   1.01600460e-02,   1.89700008e-02,
          3.78068673e-02,   2.47860539e-02,   2.32636539e-02,
          2.39265031e-02,   9.78196153e-02],
       [  5.28448124e-04,   8.02145934e-02,   1.51759208e-03,
          3.56653896e-04,   2.24039668e-02,   1.03798630e-0

I have used training - validation split to check the performance.

Taking first 5000 rows for training and others for validation

Since Neural  nets used to learn after each batch execution then abnormal batch will not result in robust minimization, here after each learning step I am using validation error for the entire set instead of ant batch to get the better picture of real world learning through this network.

In [5]:
trainx = df[:5000,:]
testX  = df[5000:,:]

Here I have designed two Neural Nets, one is single layer encoder and other one is multi layer encoder. Using Multilayer for the execution. If we use single layer the other code base will remain constant.

In [6]:
net = NeuralNet_multilayerencode()
net.double()
criterion = nn.MSELoss(size_average=True)
optimizer = optim.Adadelta(net.parameters())

Let's set parameters for deep learning execution.

We also want to record our training and validation losses for performance monitoring.

In [8]:
loss_summary = []

In [10]:
batch_size = 100
EPOCHS = 100
lr = 0.01
loss_summary = []
import time
start = time.time()
val_data = Variable(torch.from_numpy(testX))
for epoch in range(0, EPOCHS):
    optimizer.zero_grad()
    o=[]
    for x_batch in next_batch(trainx,batch_size):
        inputs = Variable(torch.from_numpy(x_batch))
        target = Variable(torch.from_numpy(x_batch))
        out = net(inputs)
        loss = criterion(out,target)
        net.zero_grad()
        loss.backward()
        for param in net.parameters():
             param.data -= lr * param.grad.data
    testout = net(val_data)
    val_loss_entire = criterion(testout, val_data)
    val_loss_entire = np.round(val_loss_entire.data.numpy()[0],5)
    training_loss = np.round(loss.data.numpy()[0],5)
    loss_summary.append([training_loss,val_loss_entire])
    print("Epoch: {}, trainingloss: {:.10f},entirevalidationloss: {:.10f}".format(epoch, training_loss,val_loss_entire))
end  = time.time()
tt = end - start
print("Timetaken:",tt)



Epoch: 0, trainingloss: 0.0141300000,entirevalidationloss: 0.0188100000
Epoch: 1, trainingloss: 0.0141200000,entirevalidationloss: 0.0188100000
Epoch: 2, trainingloss: 0.0141200000,entirevalidationloss: 0.0188000000
Epoch: 3, trainingloss: 0.0141200000,entirevalidationloss: 0.0188000000
Epoch: 4, trainingloss: 0.0141200000,entirevalidationloss: 0.0188000000
Epoch: 5, trainingloss: 0.0141100000,entirevalidationloss: 0.0188000000
Epoch: 6, trainingloss: 0.0141100000,entirevalidationloss: 0.0187900000
Epoch: 7, trainingloss: 0.0141100000,entirevalidationloss: 0.0187900000
Epoch: 8, trainingloss: 0.0141000000,entirevalidationloss: 0.0187900000
Epoch: 9, trainingloss: 0.0141000000,entirevalidationloss: 0.0187800000
Epoch: 10, trainingloss: 0.0141000000,entirevalidationloss: 0.0187800000
Epoch: 11, trainingloss: 0.0141000000,entirevalidationloss: 0.0187800000
Epoch: 12, trainingloss: 0.0140900000,entirevalidationloss: 0.0187800000
Epoch: 13, trainingloss: 0.0140900000,entirevalidationloss: 0

# We can also tweak our learning late for faster learning.